In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_interactions_train = pd.read_csv("/content/drive/MyDrive/RS Data/Assignment 2/interactions_train.csv")
df_interactions_train.head(5)

,user_id,recipe_id,date,rating,u,i
0,2046,4684,2000-02-25,5.0,22095,44367
1,2046,517,2000-02-25,5.0,22095,87844
2,1773,7435,2000-03-13,5.0,24732,138181
3,1773,278,2000-03-13,4.0,24732,93054
4,2046,3431,2000-04-07,5.0,22095,101723


In [ ]:
df_interactions_test = pd.read_csv("/content/drive/MyDrive/RS Data/Assignment 2/interactions_test.csv")
df_interactions_test.head(5)

,user_id,recipe_id,date,rating,u,i
0,8937,44551,2005-12-23,4.0,2,173538
1,56680,126118,2006-10-07,4.0,16,177847
2,349752,219596,2008-04-12,0.0,26,89896
3,628951,82783,2007-11-13,2.0,45,172637
4,92816,435013,2013-07-31,3.0,52,177935


In [ ]:
df_interactions_train.shape, df_interactions_test.shape

((698901, 6), (12455, 6))

In [ ]:
len(df_interactions_train["user_id"].unique()), len(df_interactions_test["user_id"].unique())

(25076, 12455)

In [ ]:
len(df_interactions_train["recipe_id"].unique()), len(df_interactions_test["recipe_id"].unique())

(160901, 11695)

In [ ]:
df= df_interactions_train.copy()
df1 = df_interactions_test.copy()

In [ ]:
df = df.groupby(by=['user_id','recipe_id'], as_index=False).agg({"rating":"mean"})
df1 = df1.groupby(by=['user_id','recipe_id'], as_index=False).agg({"rating":"mean"})
df.shape

(698901, 3)

In [ ]:
df.describe()

,user_id,recipe_id,rating
count,6.989010e+05,698901.000000,698901.000000
mean,1.247694e+07,156173.409849,4.574090
std,1.525031e+08,126594.880211,0.959022
min,1.533000e+03,38.000000,0.000000
25%,1.059880e+05,53169.000000,4.000000
50%,2.301020e+05,116484.000000,5.000000
75%,4.801950e+05,234516.000000,5.000000
max,2.002313e+09,537458.000000,5.000000


#### Keeping only those users who've atleast given 100 ratings.

In [ ]:
freq = df['user_id'].value_counts()
threshold = 1
mask = df['user_id'].isin(freq[freq >= threshold].index)
df = df.loc[mask]

In [ ]:
df.shape

(698901, 3)

In [ ]:
df.describe()

,user_id,recipe_id,rating
count,6.989010e+05,698901.000000,698901.000000
mean,1.247694e+07,156173.409849,4.574090
std,1.525031e+08,126594.880211,0.959022
min,1.533000e+03,38.000000,0.000000
25%,1.059880e+05,53169.000000,4.000000
50%,2.301020e+05,116484.000000,5.000000
75%,4.801950e+05,234516.000000,5.000000
max,2.002313e+09,537458.000000,5.000000


In [ ]:
len(df["user_id"].unique())

25076

In [ ]:
len(df["recipe_id"].unique())

160901

### Creating sparse Rating Matrix

In [ ]:
unique_users = df['user_id'].unique() 
unique_recipes = df['recipe_id'].unique()

In [ ]:
users_dict = {unique_users[i] : i for i in range(len(unique_users))}
recipes_dict = {unique_recipes[i] : i for i in range(len(unique_recipes))}

In [ ]:
df['user_id'] = df['user_id'].map(users_dict)
df['recipe_id'] = df['recipe_id'].map(recipes_dict)

In [ ]:
df1['user_id'] = df1['user_id'].map(users_dict)
df1['recipe_id'] = df1['recipe_id'].map(recipes_dict)

In [ ]:
df.head()

,user_id,recipe_id,rating
0,0,0,5.0
1,0,1,5.0
2,0,2,5.0
3,0,3,5.0
4,0,4,5.0


In [ ]:
df2 = df1.dropna(subset=['user_id'])
df2.shape

(12455, 3)

In [ ]:
df3 = df1.dropna(subset=['recipe_id'])
df3.shape

(0, 3)

**Here we can see that all recipees in the test do not have any prior data in train, for speculation. Thus we can't use this data for collabrative recommendation.**

In [ ]:
import pandas as pd
from scipy.sparse import coo_matrix

rating_matrix = coo_matrix((df['rating'],(df['user_id'],df['recipe_id'])))

In [ ]:
rating_matrix = rating_matrix.toarray()
rating_matrix

array([[5., 5., 5., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
rating_matrix.shape

(25076, 160901)